In [1]:
from seleniumwire import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
import time
import datetime
import json
import numpy as np
import os
import math
import random

In [2]:
class Visits:
    
    def __init__(self):
        self.is_holiday = self.is_holiday()
        self.n_daily_visits = self.n_daily_visits()
        self.halfLifeTime = 7 # Em dias
        self.duration = 60 # Em segundos
    
    
    
    def is_holiday(self):
        return False
    
    
    
    def n_daily_visits(self):
        peak = 1000 # Numero de visitas no pico da semana
        return peak
        # desv = peak * 0.05
        # now = datetime.datetime.now()
        # day_index = datetime.datetime.weekday(now)
        # # day_str = ['seg', 'ter', 'qua', 'qui', 'sex', 'sab', 'dom']
        # day_perc = [1, 0.9, 0.85, 0.8, 0.7, 0.6, 0.5]
        # daily_visits_avg = peak * day_perc[day_index]
        # if self.is_holiday:
        #     daily_visits_avg = daily_visits_avg * day_perc[6]
        # n_daily_visits = np.random.normal(loc=daily_visits_avg, scale=desv, size=1)[0]
        # n_daily_visits = round(n_daily_visits)
        # return n_daily_visits
    
    
    
    def n_hourly_visits(self):
        hour_perc = [0.008, 0.008, 0.008, 0.008, 0.015, 0.023, 0.038, 0.076, 0.092, 0.061, 0.053, 0.038, 0.069, 0.053, 0.038, 0.031, 0.038, 0.076, 0.092, 0.061, 0.046, 0.031, 0.023, 0.014]
        return hour_perc
        # now = datetime.datetime.now()
        # hour_index = now.hour
        # n_hourly_visits = self.n_daily_visits * hour_perc[hour_index]
        # return int(np.ceil(n_hourly_visits))
    
    
    
    def n_visits_finished(self):
        hour_perc = [0.008, 0.008, 0.008, 0.008, 0.015, 0.023, 0.038, 0.076, 0.092, 0.061, 0.053, 0.038, 0.069, 0.053, 0.038, 0.031, 0.038, 0.076, 0.092, 0.061, 0.046, 0.031, 0.023, 0.014]
        now = datetime.datetime.now()
        hour_index = now.hour
        n_hourly_visits = self.n_daily_visits * hour_perc[hour_index]
        return int(np.ceil(n_hourly_visits))
    
    
    
    def visitsByOriginDay(self, delta_days):
        referenceDayVisits = self.n_daily_visits * (1/2)**((delta_days+0)/self.halfLifeTime)
        previousDayVisits  = self.n_daily_visits * (1/2)**((delta_days+1)/self.halfLifeTime)
        return math.ceil(referenceDayVisits - previousDayVisits)
    
    
    
    def visitsList(self):
        result = []
        today = datetime.datetime.today()
        # Visitas por dia de referência
        for i in range(56):
            nVisitsByOriginDay = self.visitsByOriginDay(i)
            for v in range(nVisitsByOriginDay):
                visit = {
                    'origin_day': 'today' if i == 0 else (today - datetime.timedelta(days=i)).strftime("%Y%m%d"),
                    'n_in_origin_day': v+1,
                    'visit_duration': max(math.ceil(np.random.normal(loc=self.duration, scale=self.duration/5, size=1)[0]), 10),
                }
                result.append(visit)
        # Embaralhar a lista de visitas
        random.shuffle(result)
        # Adicionar a hora de início da visita
        counter = 0
        for hour in range(24):
            nVisits = math.ceil(len(result) * self.n_hourly_visits()[hour])
            for i in range(nVisits):
                try:
                    result[counter]['start_hour'] = hour
                    counter += 1
                except:
                    pass
        filepath = f'./visits/{today.strftime("%Y%m%d")}.json'
        with open(filepath, 'w') as file:
            json.dump(result, file)



    def should_start_new_visit(self, last_visit_hour):
        now = datetime.datetime.now()
        result = now.hour != last_visit_hour
        if result and now.hour == 0:
            self.visitsList()
        return result



    def get_visit_from_hour(self):
        now = datetime.datetime.now()
        filepath = f'./visits/{now.strftime("%Y%m%d")}.json'
        with open(filepath, 'r') as file:
            visits = json.load(file)
        result = [visit for visit in visits if visit['start_hour'] == now.hour]
        return result


a=Visits()
a.visitsList()


In [3]:
class BrowserBot:
    
    def __init__(self, duration, cookiePath = None):
        self.cookiePath = cookiePath
        self.cookie_exists = self.cookie_exists()
        self.driver = self.open_browser()
        self.duration = duration
        self.createdAt = self.visit()
        self.finished = False
    
    
    
    def open_browser(self, proxy=None, proxy_type="socks5"):
        proxy={
            'proxyUser':'pl-johannesnogueira_area-BR_state-sãopaulo',
            # 'proxyUser':'pl-johannesnogueira_area-BR_city-MANAUS',
            'proxyPass':'602748',
            'proxyHost':'gate-us.proxylite.com',
            'proxyPort':'9595'
        }
        seleniumwire_options = {}
        if proxy:
            proxy_host = proxy['proxyHost']
            proxy_port = proxy['proxyPort']
            proxy_user = proxy['proxyUser']
            proxy_pass = proxy['proxyPass']
            seleniumwire_options = {
                'proxy': {
                    'http': f"{proxy_type}://{proxy_user}:{proxy_pass}@{proxy_host}:{proxy_port}",
                    'https': f"{proxy_type}://{proxy_user}:{proxy_pass}@{proxy_host}:{proxy_port}",
                    'no_proxy': 'localhost,127.0.0.1'  # URLs que não usarão proxy
                }
            }
            chrome_options = Options()
            chrome_options.add_argument('--disable-blink-features=AutomationControlled')
            chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
            chrome_options.add_experimental_option('useAutomationExtension', False)
            chrome_options.add_argument('--ignore-certificate-errors')
            chrome_options.add_argument('--allow-insecure-localhost')
            chrome_options.add_argument('--allow-running-insecure-content')
            # chrome_options.add_argument('--headless')  # Se desejar rodar em modo headless

            # Inicializar o WebDriver com Selenium Wire
            driver = webdriver.Chrome(
                service=Service(ChromeDriverManager().install()),
                options=chrome_options,
                seleniumwire_options=seleniumwire_options
            )
            
            # Configurar o WebDriver para evitar detecção
            driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            
        if self.cookie_exists:
            with open(self.cookiePath, 'r') as file:
                cookies = json.load(file)
                for cookie in cookies:
                    driver.add_cookie(cookie)
        return driver
    
    
    
    def visit(self):
        if self.cookie_exists:
            self.loadCookie(self.cookiePath)
        # self.driver.get("https://api.ip.cc/")
        self.driver.get("https://jornalsocrates.com.br/")
        # return time.time()
    
    
    
    def cookie_exists(self):
        try:
            with open(self.cookiePath, 'r'):
                pass
            return True
        except FileNotFoundError:
            return False
    
    
    
    def saveCookie(self):
        try:
            n_files = 0
            if not self.cookie_exists:
                now = datetime.datetime.now()
                # Criar o folder, caso não exista - Significa que deve ser criado um novo arquivo na pasta de hoje
                folderPath = f'./cookies/{now.strftime("%Y%m%d")}'
                if not os.path.exists(folderPath):
                    try:
                        os.makedirs(folderPath)
                    except OSError as e:
                        print(f"Erro ao criar a pasta {folderPath}: {e}")
                # Contabiliza a quantidad ede arquivos já existente no arquivo
                for raiz, dirs, arquivos in os.walk(folderPath):
                    n_files = len(arquivos)
                # Salva o arquivo de cookies
                filePath = f"{folderPath}/{n_files + 1}.json"
                os.makedirs(os.path.dirname(filePath), exist_ok=True)
                cookies = self.driver.get_cookies()
                with open(filePath, 'w') as f:
                    json.dump(cookies, f)
        except Exception as e:
            print(f"Erro ao salvar cookies: {e}")
    
    
    
    def loadCookie(self, cookiePath):
        try:
            with open(cookiePath, 'r') as file:
                cookies = json.load(file)
                for cookie in cookies:
                    if 'name' in cookie and 'value' in cookie:
                        cookie.pop('sameSite', None)
                        self.driver.add_cookie(cookie)
        except Exception as e:
            print(f"Erro ao carregar cookies: {e}")
    
    
    
    def closeIfTime(self):
        if time.time() - self.createdAt > self.duration:
            if not self.finished: 
                print('Interação 1 iniciada')
                self.driver.get("https://jornalsocrates.com.br/noticia/66e07f5cc11b77d9375fff81")
                self.finished = True
                print('Interação 1 finalizada')
            else: 
                print('Interação 2 iniciada')
                self.saveCookie()
                self.driver.quit()
                print('Interação 2 finalizada')


In [4]:
class HourIteration:
    def __init__(self, visits_list):
        self.total_visits = len(visits_list)
        self.visits_list = visits_list
        self.max_drivers = 10
        self.n_opened = 0
        self.n_visits = 0
        self.iteration = 0
        self.browsers = []

        while self.n_visits < self.total_visits:
            # Garantir a quantidade máxima de janelas abertas
            while (self.n_opened < self.max_drivers) and (self.n_visits < self.total_visits):
                try:
                    visit = visits_list[0]
                    cookiePath = f'./cookies/{visit["origin_day"]}/{visit["n_in_origin_day"]}.json'
                    driver_duration = visit['visit_duration']
                    del visits_list[0]
                except:
                    cookiePath = f'./cookies/fake/cookie_fake.json'
                try:
                    bot = BrowserBot(driver_duration, cookiePath)
                    self.browsers.append(bot)
                except:
                    print('Erro ao abrir o navegador')
                self.n_opened = len(self.browsers)
            
            for b in self.browsers:
                if b.finished:
                    b.closeIfTime()
                    self.browsers.remove(b)
                    self.n_visits += 1
                else: 
                    try:
                        b.closeIfTime()
                    except Exception as e:
                        print(f'Erro ao fechar o navegador AQUI')
            
            self.n_opened = len(self.browsers)

            time.sleep(5)
            self.iteration += 1

        for b in self.browsers:
            try:
                b.driver.quit()
            except:
                print('Erro ao fechar o navegador na última etapa')



In [5]:
class TrafficBot:
    def __init__(self):
        last_visit_hour = -1
        visits = Visits()
        while True:
            if Visits().should_start_new_visit(last_visit_hour):
                startedAt = datetime.datetime.now()
                # last_visit_hour = startedAt.hour
                hourly_visits = visits.get_visit_from_hour()
                print(f'{startedAt.year}-{str(startedAt.month).rjust(2, '0')}-{str(startedAt.day).rjust(2, '0')}-{str(startedAt.hour).rjust(2, '0')}:{str(startedAt.minute).rjust(2, '0')}:{str(startedAt.second).rjust(2, '0')} >> New visits trigger with {str(len(hourly_visits)).rjust(5)} visitors')
                HourIteration(hourly_visits)
                finishedAt = datetime.datetime.now()
                print(f'{finishedAt.year}-{str(finishedAt.month).rjust(2, '0')}-{str(finishedAt.day).rjust(2, '0')}-{str(finishedAt.hour).rjust(2, '0')}:{str(finishedAt.minute).rjust(2, '0')}:{str(finishedAt.second).rjust(2, '0')} >> Visitas finalizadas')
            else:
                time.sleep(60)


In [6]:
TrafficBot()

2024-09-19-17:53:26 >> New visits trigger with    78 visitors
Erro ao abrir o navegador


In [ ]:
# import requests
# pconfig={
#     'proxyUser':'pl-johannesnogueira_area-BR_city-MANAUS',
#     'proxyPass':'602748',
#     'proxyHost':'gate-us.proxylite.com',
#     'proxyPort':'9595'
# }
# url = "https://api.ip.cc/"
# proxies = {
#   "http": "http://{}:{}@{}:{}".format(pconfig['proxyUser'], pconfig['proxyPass'], pconfig['proxyHost'], pconfig['proxyPort']),
#   "https": "http://{}:{}@{}:{}".format(pconfig['proxyUser'], pconfig['proxyPass'], pconfig['proxyHost'], pconfig['proxyPort'])
# }
# result = requests.get(url=url, proxies=proxies)
# print(result.text)

# a = BrowserBot(60, './oi')
